# GIST Driver
##Test on events of interest:
  
  texnet2023vqsh

  
Assumes InjectionSQLScheduled completed

#Configuration

##Libraries

In [0]:
! pip install geopandas
! pip install geodatasets
! pip install folium matplotlib mapclassify contextily

     |████████████████████████████████| 1.1 MB 15.6 MB/s 
     |███

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 20.6 MB 38.3 MB/s 
     |████████████████████████████████| 125 kB 65.1 MB/s 
     |████████████████████████████████| 40 kB 1.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Not uninstalling numpy at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-a18a3a54-e1ea-486e-9b6b-7e01d63f49e7
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Not uninstalling scipy at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-a18a3a54-e1ea-486e-9b6b-7e01d63f49e7
    Can't uninstall 'scipy'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral

In [0]:
import sys
basePath='/Workspace/Users/bill.curry@exxonmobil.com'
gistPath='/Workspace/Users/bill.curry@exxonmobil.com/gist'
sys.path.append(basePath)
import dataBricksConfig as db

In [0]:
import numpy as np
import pandas as pd
import os
import gistMCLive as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
import contextily as cx

/local_disk0/.ephemeral_nfs/envs/pythonEnv-a18a3a54-e1ea-486e-9b6b-7e01d63f49e7/lib/python3.9/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:
import gistPlots as gp

## Parameters

### Input Files
Produced from injectionSQLScheduled.py

In [0]:
injPath=gistPath+'/ScheduledOutput'

DeepWellFile=injPath+'/deep.csv'
DeepInjFile=injPath+'/deepReg.csv'
ShallowWellFile=injPath+'/shallow.csv'
ShallowInjFile=injPath+'/shallowReg.csv'
runPath=gistPath+'/Outputs/'

################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runPath+'MC'

###General

In [0]:
######################
# General Parameters #
######################
nRealizations=500
nTimeBins=21
WaterDensity=1020.
WaterViscosity=0.001
VerticalCompressibility=0.000000001
FluidCompressibility=0.00000000036
FaultFrictionCoeff=0.6
RockFrictionCoeff=0.6

###################################
# General Parameter Uncertainties #
###################################
WaterDensityUncertainty=5.
WaterViscosityUncertainty=0.000001
VerticalCompressibilityUncertainty=0.00000000001
FluidCompressibilityUncertainty=0.000000000001
FaultFrictionCoeffUncertainty=0.05
RockFrictionCoeffUncertainty=0.05

###Shallow interval

In [0]:
###############################
# Shallow Interval Parameters #
###############################
ShallowPorosityPercent=15.
ShallowPermMD=10.
ShallowThicknessFT=3700.
# From Lei's excel sheet for Stanton - Young's modulus 5.83 GPA
# Shear = Young / 2(1+ Poisson's ratio)
ShallowShearModulus=2.4e9
# Is Dynamic undrained?
ShallowPoissonsRatioDrained=0.19
ShallowPoissonsRatioUndrained=0.215
# No idea - probably needs updating
ShallowBiotsCoefficient=0.31

##################################
# Shallow Interval Uncertainties #
##################################
ShallowPorosityPercentUncertainty=8.
# I think this should be a flat distribution bounds set using the logarithm of the permeability - now implemented
ShallowPermLogMDUncertainty=1.
ShallowThicknessFTUncertainty=500.
ShallowShearModulusUncertainty=1e9
ShallowPoissonsRatioDrainedUncertainty=0.01
ShallowPoissonsRatioUndrainedUncertainty=0.01
ShallowBiotsCoefficientUncertainty=0.05

### Deep Interval

In [0]:
###############################
# Deep Interval Parameters #
###############################
DeepPorosityPercent=8.
# Pulse pressure test shows background diffusivity of 0.5 m2/s
# Back that into permeaiblity: k = D * eta * (Cr + porosity Cp)
# 0.5 x 0.001 x (1e -9 + 0.08 x 3.6 e-10) = 5.144 e-13
# / 9.9e-16 m2 to mD = 520mD!
DeepPermMD=100.
# Now let's change this to something where 1D is the high range and 10mD is the low range

DeepThicknessFT=1400.
# Roughly double of shallow - 
DeepShearModulus=5e9
DeepPoissonsRatioDrained=0.30
DeepPoissonsRatioUndrained=0.31
DeepBiotsCoefficient=0.31

##################################
# DeepInterval Uncertainties #
##################################
DeepPorosityPercentUncertainty=5.
DeepPermLogMDUncertainty=1.
DeepThicknessFTUncertainty=200.
DeepShearModulusUncertainty=1e9
DeepPoissonsRatioDrainedUncertainty=0.005
DeepPoissonsRatioUndrainedUncertainty=0.005
DeepBiotsCoefficientUncertainty=0.05

#Earthquake Selection

In [0]:
eqs=es.eqSQL()
EQDF=eqs.getEarthquake('texnet2023vqsh')
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# What about the other fault plane solution?
EQDF['Strike']=216.
EQDF['Dip']=65.
EQDF['Rake']=-127.
# We could make use of the Rake
EQDF.info()
eq=EQDF.loc[0]
EQDF.to_csv(runPath+'/11-4-23EQ.csv')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              1 non-null      int64         
 1   DataSource                  1 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     1 non-null      object        
 4   EventTimeUtc                1 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    1 non-null      object        
 6   EventTimeZone               1 non-null      object        
 7   EventType                   1 non-null      object        
 8   DepthKm                     1 non-null      float64       
 9   DepthErrorKm                1 non-null      float64       
 10  Magnitude                   1 non-null      float64       
 11  MagnitudeError              0 non-null      float64   

#GIST Initialization

## Shallow 

In [0]:
########################################
# Initialize GIST for shallow interval #
########################################
shallowM=gi.gistMC(nReal=nRealizations,
                   ntBin=nTimeBins)

In [0]:
##############################################################
# Initialize pore pressure model                             #
# Parameters mostly from Katie Smye, are rough at this point #
##############################################################
shallowM.initPP(rho_0=WaterDensity,
                nta=WaterViscosity,
                phi=ShallowPorosityPercent,
                kMD=ShallowPermMD,
                h=ShallowThicknessFT,
                alphav=VerticalCompressibility,
                beta=FluidCompressibility)
######################
# Uncertainty ranges #
######################
shallowM.initMCPP(rhoUnc=WaterDensityUncertainty,
                  ntaUnc=WaterViscosityUncertainty,
                  phiUnc=ShallowPorosityPercentUncertainty,
                  kMDLogUnc=ShallowPermLogMDUncertainty,
                  hUnc=ShallowThicknessFTUncertainty,
                  alphavUnc=VerticalCompressibilityUncertainty,
                  betaUnc=FluidCompressibilityUncertainty)

 gistMC.initMCPP - kMD original Central value, upper bound, lower bound, updated value, uncertainty: 10.0 100.0 1.0 50.5 49.5


In [0]:
######################################################
# These parameters need more vetting for Tarzan area #
######################################################
shallowM.initPE(mu=ShallowShearModulus,
                nu=ShallowPoissonsRatioDrained,
                nu_u=ShallowPoissonsRatioUndrained,
                alpha=ShallowBiotsCoefficient,
                mu_f=FaultFrictionCoeff,
                mu_r=RockFrictionCoeff,
                match=True)
######################
# Uncertainty ranges #
######################
shallowM.initMCPE(muUnc=ShallowShearModulusUncertainty,
                  nuUnc=ShallowPoissonsRatioDrainedUncertainty,
                  nu_uUnc=ShallowPoissonsRatioUndrainedUncertainty,
                  alphaUnc=ShallowBiotsCoefficientUncertainty,
                  muFUnc=FaultFrictionCoeffUncertainty,
                  muRUnc=RockFrictionCoeffUncertainty,
                  match=True)

## Deep

In [0]:
##################################################
# Initialize 2nd GIST instance for deep interval #
##################################################
deepM=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
##################################################################
# Initialize pore pressure model                                 #
##################################################################
deepM.initPP(rho_0=WaterDensity,
             phi=DeepPorosityPercent,
             kMD=DeepPermMD,
             h=DeepThicknessFT,
             alphav=VerticalCompressibility,
             beta=FluidCompressibility)

deepM.initMCPP(rhoUnc=WaterDensityUncertainty,
               ntaUnc=WaterViscosityUncertainty,
               phiUnc=DeepPorosityPercentUncertainty,
               kMDLogUnc=DeepPermLogMDUncertainty,
               hUnc=DeepThicknessFTUncertainty,
               alphavUnc=VerticalCompressibilityUncertainty,
               betaUnc=FluidCompressibilityUncertainty)

 gistMC.initMCPP - kMD original Central value, upper bound, lower bound, updated value, uncertainty: 100.0 1000.0 10.0 505.0 495.0


In [0]:
######################################################
# These parameters need more vetting for Tarzan area #
######################################################
deepM.initPE(mu=DeepShearModulus,
             nu=DeepPoissonsRatioDrained,
             nu_u=DeepPoissonsRatioUndrained,
             alpha=DeepBiotsCoefficient,
             mu_f=FaultFrictionCoeff,
             mu_r=RockFrictionCoeff,
             match=True)
deepM.initMCPE(muUnc=DeepShearModulusUncertainty,
               nuUnc=DeepPoissonsRatioDrainedUncertainty,
               nu_uUnc=DeepPoissonsRatioUndrainedUncertainty,
               alphaUnc=DeepBiotsCoefficientUncertainty,
               muFUnc=FaultFrictionCoeffUncertainty,
               muRUnc=RockFrictionCoeffUncertainty,
               match=True)

In [0]:
#######################################
# Add uncertainties to both intervals #
#######################################

######################
# Post diffusivities #
######################
print("Shallow Diffusivity="+str(shallowM.diffPPMax))
print("Deep Diffusivity="+str(deepM.diffPPMax))


Shallow Diffusivity=0.0952405163641887
Deep Diffusivity=0.9719884689589782


#Well selection

####Input wells

In [0]:
#########################################
# Add potential wells to each instance  #
#########################################
print("Before adding shallow wells")
shallowM.addWells(ShallowWellFile,ShallowInjFile,verbose=1)

Before adding shallow wells
 gistMC.addWells: well file added with  5050  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'APINumber', 'UICNumber', 'Basin', 'County',
       'SurfaceHoleLatitude', 'SurfaceHoleLongitude', 'WellName', 'WellNumber',
       'InjectionType', 'CompletedWellDepthClassification', 'InjectionStatus',
       'CompletionAndDrillingPermitStatus', 'StartDate', 'PermitIsAmendment',
       'DaysToPermitApproval', 'PermittedMaxLiquidBPD',
       'PermittedMaxLiquidPSIG', 'PermittedIntervalBottomFt',
       'PermittedIntervalTopFt', 'PermittedCommercialStatus',
       'PermittedWellDepthClassification', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType',
       'PermitStage', 'SRAOrSIR', 'UploadTime'],
      dtype='object')
 gistMC.addWells: injection file has  4100  unique wells
 gistMC.addWells: injection file first day:  3660.0
 gistMC.addWells: injection file last day:  19700.0
 gistMC.addWells: inject

In [0]:
print("Before adding deep wells")
deepM.addWells(DeepWellFile,DeepInjFile,verbose=1)

Before adding deep wells
 gistMC.addWells: well file added with  2123  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'APINumber', 'UICNumber', 'Basin', 'County',
       'SurfaceHoleLatitude', 'SurfaceHoleLongitude', 'WellName', 'WellNumber',
       'InjectionType', 'CompletedWellDepthClassification', 'InjectionStatus',
       'CompletionAndDrillingPermitStatus', 'StartDate', 'PermitIsAmendment',
       'DaysToPermitApproval', 'PermittedMaxLiquidBPD',
       'PermittedMaxLiquidPSIG', 'PermittedIntervalBottomFt',
       'PermittedIntervalTopFt', 'PermittedCommercialStatus',
       'PermittedWellDepthClassification', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType',
       'PermitStage', 'SRAOrSIR', 'UploadTime'],
      dtype='object')
 gistMC.addWells: injection file has  1780  unique wells
 gistMC.addWells: injection file first day:  3880.0
 gistMC.addWells: injection file last day:  19700.0
 gistMC.addWells: injection

In [0]:
shallowM.wellDF.info()
EQDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 5050 non-null   int64  
 1   InjectionWellId                    5050 non-null   int64  
 2   APINumber                          5050 non-null   object 
 3   UICNumber                          4389 non-null   object 
 4   Basin                              3878 non-null   object 
 5   County                             5050 non-null   object 
 6   SurfaceHoleLatitude                5050 non-null   float64
 7   SurfaceHoleLongitude               5050 non-null   float64
 8   WellName                           5050 non-null   object 
 9   WellNumber                         5050 non-null   object 
 10  InjectionType                      5050 non-null   object 
 11  CompletedWellDepthClassification   4792 non-null   objec

In [0]:
shallowPPWells,shallowPPInj=shallowM.findWells(eq,PE=False)
shallowPEWells,shallowPEInj=shallowM.findWells(eq,PE=True)
shallowPPWells['EventID']=eq['EventID']
shallowPEWells['EventID']=eq['EventID']
deepPPWells,deepPPInj=deepM.findWells(eq,PE=False)
deepPEWells,deepPEInj=deepM.findWells(eq,PE=True)
deepPPWells['EventID']=eq['EventID']
deepPEWells['EventID']=eq['EventID']

In [0]:
#shallowPPWells.to_csv(runPath+'/Stanton11-4-23_ShallowPPWells.csv')
#shallowPEWells.to_csv(runPath+'/Stanton11-4-23_ShallowPEWells.csv')
#deepPPWells.to_csv(runPath+'/Stanton11-4-23_DeepPPWells.csv')
#deepPEWells.to_csv(runPath+'/Stanton11-4-23_DeepPEWells.csv')

In [0]:
# Dump injection data to file:
#deepPPInj.to_csv(runPath+'/Stanton11-4-23_DeepPPInj.csv')
#shallowPPInj.to_csv(runPath+'/Stanton11-4-23_ShallowPPInj.csv')

In [0]:
eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])

<command-4459818756533517>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])


In [0]:
#import gistPlots as gp

#gp.wellInjPlot(deepPPInj,deepPPWells,'01-01-2018','02-11-2023','Deep Injection',True)
#gp.wellInjPlot(shallowPPInj,shallowPPWells,'01-01-2018','02-11-2023','Shallow Injection',True)

#fig, ax = plt.subplots(figsize=(12,24))
# Merge well information onto injection rates
# Need to incorporate time sampling other than 10 day


#Initial GIST runs


In [0]:
# Run deep scenarios
#eq['Origin Date']=pd.to_datetime(eq['Origin Date'])
#dWellPPScenarios,dPPTimeSeries=deepM.runPressureScenarios(eq,deepPPWells,deepPPInj,verbose=1)

 gistMC.runPressureScenarios: Number of wells considered:  579
 gistMC.pressureScenario: well  0  of  579 :  1003681
 gistMC.pressureScenario: well  10  of  579 :  1003116
 gistMC.pressureScenario: well  20  of  579 :  1005811
 gistMC.pressureScenario: well  30  of  579 :  1004578
 gistMC.pressureScenario: well  40  of  579 :  1006462
 gistMC.pressureScenario: well  50  of  579 :  2013104
 gistMC.pressureScenario: well  60  of  579 :  2083672
 gistMC.pressureScenario: well  70  of  579 :  2100310
 gistMC.pressureScenario: well  80  of  579 :  2117806
 gistMC.pressureScenario: well  90  of  579 :  2111821
 gistMC.pressureScenario: well  100  of  579 :  2112227
 gistMC.pressureScenario: well  110  of  579 :  2079810
 gistMC.pressureScenario: well  120  of  579 :  2045831
 gistMC.pressureScenario: well  130  of  579 :  2012806
 gistMC.pressureScenario: well  140  of  579 :  2049794
 gistMC.pressureScenario: well  150  of  579 :  2119747
 gistMC.pressureScenario: well  160  of  579 :  2111

In [0]:
#sWellPPScenarios,sPPTimeSeries=shallowM.runPressureScenarios(eq,shallowPPWells,shallowPPInj,verbose=1)

 gistMC.runPressureScenarios: Number of wells considered:  130
 gistMC.pressureScenario: well  0  of  130 :  2078653
 gistMC.pressureScenario: well  10  of  130 :  2103314
 gistMC.pressureScenario: well  20  of  130 :  2018969
 gistMC.pressureScenario: well  30  of  130 :  2102379
 gistMC.pressureScenario: well  40  of  130 :  2059622
 gistMC.pressureScenario: well  50  of  130 :  2119567
 gistMC.pressureScenario: well  60  of  130 :  2061545
 gistMC.pressureScenario: well  70  of  130 :  2019097
 gistMC.pressureScenario: well  80  of  130 :  2116214
 gistMC.pressureScenario: well  90  of  130 :  2114686
 gistMC.pressureScenario: well  100  of  130 :  2071313
 gistMC.pressureScenario: well  110  of  130 :  2104114
 gistMC.pressureScenario: well  120  of  130 :  2097492
 gistMC.pressureScenario: scenario  1  of  500 : Max P: 10.00647156708976
 gistMC.pressureScenario: scenario  2  of  500 : Max P: 11.815003865639992
 gistMC.pressureScenario: scenario  3  of  500 : Max P: 8.5786917235944

In [0]:

baseOutPath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/Stanton11-4-23_v2_'
#dWellPPScenarios.to_csv(baseOutPath+'deepPP.csv')
#sWellPPScenarios.to_csv(baseOutPath+'shallowPP.csv')

In [0]:
#dPPTimeSeriesSum=dPPTimeSeries.sum(axis=1)
#sPPTimeSeriesSum=sPPTimeSeries.sum(axis=1)
dPPTimes=np.linspace(deepM.injOT, deepM.injOT+deepM.injDT*(deepM.injNT-1.),deepM.injNT)
sPPTimes=np.linspace(shallowM.injOT, shallowM.injOT+shallowM.injDT*(shallowM.injNT-1.),shallowM.injNT)

In [0]:
np.save(baseOutPath+'deepPPTimes.npy',dPPTimes)
np.save(baseOutPath+'shallowPPTimes.npy',sPPTimes)
#np.save(baseOutPath+'shallowPPTimeSeriesSum.npy',sPPTimeSeriesSum)
#np.save(baseOutPath+'deepPPTimeSeriesSum.npy',dPPTimeSeriesSum)

In [0]:
# Window out significant wells - save individual time series for those


In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
shallowWindowedWells=sWellPPScenarios[sWellPPScenarios['Pressures']>0.1]['ID'].unique()

In [0]:
# dPPTimeSeries is time, well, realization - need top wells with time series of pressures
dPPWellIndices=deepPPWells['ID'].isin(deepWindowedWells)
print(sum(dPPWellIndices),' wells out of ',len(dPPWellIndices),' have >0.1 PSI contribution')
dPPTimeSeriesSelected=dPPTimeSeries[:,dPPWellIndices,:]
dPPSelectedWellNameDF=deepPPWells['WellName'][dPPWellIndices]
print(dPPSelectedWellNameDF.to_list())

In [0]:
i=0
for wellName in dPPSelectedWellNameDF.to_list():
  np.save(baseOutPath+'deepPPTimeSeries'+wellName+'.npy',dPPTimeSeriesSelected[:,i,:])
  i=i+1
  print('Well '+wellName+' written '+i)
dPPSelectedWellNameDF.to_csv(baseOutPath+'deepSelectedWellNames.csv')

In [0]:
# sPPTimeSeries is time, well, realization - need top wells with time series of pressures
sPPWellIndices=shallowPPWells['ID'].isin(shallowWindowedWells)
print(sum(sPPWellIndices),' wells out of ',len(sPPWellIndices),' have >0.1 PSI contribution')
sPPTimeSeriesSelected=sPPTimeSeries[:,sPPWellIndices,:]
sPPSelectedWellNameDF=shallowPPWells['WellName'][sPPWellIndices]
print(sPPSelectedWellNameDF.to_list())


In [0]:
i=0
for wellName in sPPSelectedWellNameDF.to_list():
  np.save(baseOutPath+'shallowPPTimeSeries'+wellName+'.npy',sPPTimeSeriesSelected[:,i,:])
  i=i+1
  print('Well '+wellName+' written '+str(i))
sPPSelectedWellNameDF.to_csv(baseOutPath+'shallowSelectedWellNames.csv')

In [0]:
dWellPEScenarios=deepM.runPoroelasticScenarios(eq,deepPEWells,deepPEInj,verbose=1)

In [0]:
sWellPEScenarios=shallowM.runPoroelasticScenarios(eq,shallowPEWells,shallowPEInj,verbose=1)

In [0]:

dWellPEScenarios.to_csv(baseOutPath+'deepPE.csv')

In [0]:

sWellPEScenarios[sWellPEScenarios['Realization']<250].to_csv(baseOutPath+'shallowPE0.csv')
sWellPEScenarios[sWellPEScenarios['Realization']>250].to_csv(baseOutPath+'shallowPE1.csv')


In [0]:
sWellPEScenarios.info()

In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
maxPressureList=[]
for ID in deepWindowedWells:
  maxPressure=max(filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtDWellPPScenarios['MaxPressure']=maxPressureList
filtDWellPPScenarios=filtDWellPPScenarios.sort_values(by='MaxPressure',ascending=False)
shallowWindowedWells=sWellPPScenarios[sWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtSWellPPScenarios=sWellPPScenarios[sWellPPScenarios['ID'].isin(shallowWindowedWells)]
maxPressureList=[]
for ID in shallowWindowedWells:
  maxPressure=max(filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtSWellPPScenarios['MaxPressure']=maxPressureList
filtSWellPPScenarios=filtSWellPPScenarios.sort_values(by='MaxPressure',ascending=False)

In [0]:
filtDWellPPScenarios=filtDWellPPScenarios.merge(deepPPWells,on='ID')
filtSWellPPScenarios=filtSWellPPScenarios.merge(shallowPPWells,on='ID')
print('deep ',filtDWellPPScenarios.shape[0],', shallow',filtSWellPPScenarios.shape[0])

## Save Results

In [0]:
# Save results - visualize separately!
baseOutPath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/Stanton11-4-23_v2_'
#dWellPPScenarios.to_csv(baseOutPath+'deepPP.csv')
dWellPEScenarios.to_csv(baseOutPath+'deepPE.csv')
#sWellPPScenarios.to_csv(baseOutPath+'shallowPP.csv')
sWellPEScenarios.to_csv(baseOutPath+'shallowPE.csv')
#np.save(baseOutPath+'deepPPTimeSeries0.csv',dPPTimeSeries['Realization'<100])
#np.save(baseOutPath+'shallowPPTimeSeries.csv',sPPTimeSeries)

## Sanity check of initial results
### Check injection on wells with non-negligable pressure contributions

In [0]:
# Select injection histories of wells
deepWindowedInjection=deepPPInj[deepPPInj['ID'].is_in(deepWindowedWells)]
shallowWindowedInjection=shallowPPInj[shallowPPInj['ID'].is_in(shallowWindowedWells)]
deepInjQC=deepWindowedInjection.merge(deepPPWells[['Name','Operator','APINumber','Latitude','Longitude']],on='ID',how='left')
shallowInjQC=shallowWindowedInjection.merge(shallowPPWells[['Name','Operator','APINumber','Latitude','Longitude']],on='ID',how='left')

In [0]:
# Make stack plots of recent injection - Deep
injQC=deepInjQC
injQC['Date']=pd.to_datetime(injQC.Date)
# Generate x and y
dates=injQC['Date'].drop_duplicates()
# Loop over wells
oDate=min(dates)
eDate=max(dates)
dateRange=pd.date_range(start=oDate,end=eDate, freq='10D')
# Get x min, x max
wellIDs=injQC['ID'].drop_duplicates()
allWellBPDs=[]
allWellLabels=[]
for well in wellIDs:
  wellDates=injQC[injQC['ID']==well]['Date']
  wellBPDs=injQC[injQC['ID']==well]['BPD']
  minWD=min(wellDates)
  maxWD=max(wellDates)
  nPrepend=sum(dateRange<minWD)
  nAppend=sum(dateRange>maxWD)
  #print('Well ',well,' ',minWD,maxWD,oDate,eDate,nPrepend,' prePad, ',nAppend,' postPad, ',len(dateRange))
  #print('allWellBPDs: ',allWellBPDs)
  if nPrepend>0:
    prependZeroBPDs=pd.Series(np.zeros([nPrepend]))
  else:
    prependZeroBPDs=None
  if nAppend>0:
    appendZeroBPDs=pd.Series(np.zeros([nAppend,1]))
  else:
    appendZeroBPDs=None
  padWellBPDs=pd.concat([prependZeroBPDs,wellBPDs,appendZeroBPDs])
  allWellBPDs.append(padWellBPDs)
  #allWellLabels.append(injQC['WellName'][injQC['ID']==well][0])
  allWellLabels.append(injQC['WellName'][injQC['ID']==well].loc[0])
# Now have allWellBPDs, allWellLabels, and dateRange all set
# Generate Stacked area plot of injection
# y needs to be defined for each well, and extended for the full time series
col = sns.color_palette("hls", len(wellIDs))
#zoomDate=oneEQ['Origin Date'] - pd.DateOffset(years=2)
ax.stackplot(dateRange,allWellBPDs, labels=allWellLabels,colors=col)
ax.set_title('Selected well injection')
ax.set_xlabel('Date')
ax.set_ylabel('Rate (BPD)')
#if startDate!=None or endDate!=None: axes.xlim(startDate,endDate)
ax.legend(loc='upper left')
plt.show()

In [0]:
# Repeat for Shallow

In [0]:
# Plot Time Series of total pressures and injection histories


In [0]:

print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:

print(shallowWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:
# Concatenate pore pressure and poroelastic results
dWellPPScenarios['Physics']='Pore Pressure'
dWellPEScenarios['Physics']='Poroelastic'
dWellPPScenarios['Stresses']=dWellPPScenarios['Pressures']
allDeep=pd.concat(objs=[dWellPPScenarios,dWellPEScenarios[dWellPEScenarios['ID'].isin(dWellPPScenarios['ID'])]])
allDeep=allDeep.merge(deepPEWells,on='ID')



In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:
maxPressureList=[]
for ID in deepWindowedWells:
  maxPressure=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures']
  maxPressureList.append
dWellPPScenarios['MaxPressure']=dWellPPScenarios['ID'].map(maxPressureDict)
dWellPPScenarios=dWellPPScenarios.sort_values(by='MaxPressure',ascending=False)

In [0]:
# Select wells that only have greater than a 0.1 psi stress increase
deepWindowedWells=allDeep[allDeep['Stresses']>0.1]['ID'].unique()
allDeepWindowed=allDeep[allDeep['ID'].isin(deepWindowedWells)]
# Get maximum pressure or stress for each well, sort by that
maxList=[]
for ID in deepWindowedWells:
  # get maximum row
  maxRow=allDeepWindowed[allDeepWindowed['ID']==ID]['Stresses'].idxmax()
  maxList.append([allDeepWindowed.iloc[maxRow]['ID'],allDeepWindowed.iloc[maxRow]['Stresses']])
wellOrdered=pd.DataFrame(maxList,columns=['ID','Stresses']).sort_values(by='Stresses')
IDOrder=wellOrdered['ID'].to_list()
sortedDeepWindowed=allDeepWindowed.sort_values(by="ID", key=lambda column: column.map(lambda e: IDOrder.index(e)), inplace=True)

In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(30,200))
# Relative contributions
sns.stripplot(data=allDeep,x='Stresses',y='WellName',hue='Physics',dodge=True,jitter=True,alpha=0.1,ax=axes)
axes.set_title('Deep Well Stresses')
axes.set_xlabel('Stress (PSI)')

In [0]:
# Concatenate pore pressure and poroelastic results
sWellPPScenarios['Physics']='Pore Pressure'
sWellPEScenarios['Physics']='Poroelastic'
sWellPPScenarios['Stresses']=sWellPPScenarios['Pressures']
allShallow=pd.concat(objs=[sWellPPScenarios,sWellPEScenarios[sWellPEScenarios['ID'].isin(sWellPPScenarios['ID'])]])
allShallow=allShallow.merge(shallowPEWells,on='ID')


In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(30,10))
# Relative contributions
sns.stripplot(data=allShallow,x='Stresses',y='WellName',hue='Physics',dodge=True,jitter=True,alpha=0.1,ax=axes)
axes.set_title('Shallow Well Stresses')
axes.set_xlabel('Stress (PSI)')

In [0]:
# Plot results
fig,axes=plt.subplots(3,1, sharex=True, figsize=(60,20))
gp.disaggregationPlot(axes[0],dWellPPScenarios,dWellPEScenarios,deepPEWells,'Deep')
gp.disaggregationPlot(axes[1],sWellPPScenarios,sWellPEScenarios,shallowPEWells,'Shallow')
# Concatenate pore pressure and poroelastic results


In [0]:
# 

dayDVec=np.linspace(deepM.injOT, deepM.injOT+deepM.injDT*(deepM.injNT-1.),deepM.injNT)
dPPTimeSeriesSum=np.sum(dPPTimeSeries,axis=1)
realization=np.linspace(0,499,500)
nwC=deepPPWells.shape[0]
dTPPDF=pd.DataFrame(columns=['Days','Realization','TotalPressure'])
dPPDF=pd.DataFrame(columns=['Days','Realization','ID','Pressure','TotalPressure'])
dTPPDF['Days']=np.tile(dayDVec,deepM.nReal)
dTPPDF['TotalPressure']=np.reshape(dPPTimeSeriesSum[:,:],deepM.injNT*deepM.nReal,order='F')
dTPPDF['Realization']=np.tile(realization,deepM.injNT)
dTPPDF=gi.getDates(dTPPDF,deepM.epoch)

In [0]:
fig,axes=plt.subplots(2,1, sharex=True, figsize=(40,15))

sns.lineplot(ax=axes[0],x='Date',y='TotalPressure',hue='Realization',data=dTPPDF)
sns.scatterplot(data=filtEQs,ax=axes[0],x='Date',y='Magnitude',s=300,marker='*')
axes[0].set_title('Deep Pressure Histories')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Pressure (PSI)')
sns.lineplot(ax=axes[1],x='Date',y='TotalPressure',hue='Realization',data=sTPPDF)
sns.scatterplot(data=filtEQs,ax=axes[1],x='Date',y='Magnitude',s=300,marker='*')
plt.show()